In [ ]:
!pip install ultralytics

# Used Chat GPT and Google Colab AI
# Used the in-class code as well

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
model = YOLO("yolov8m.pt")

In [ ]:
import os
from glob import glob

# Load image path and allow for multiple extensions
image_path = '/content/'
image_extensions = ['*.jpg', '*.jpeg', '*.png']

# Get all image files in the directory - already added into Colab
images = []
for ext in image_extensions:
    images.extend(glob(os.path.join(image_path, ext)))


In [ ]:
i = 0
for img in images:
  results = model.predict(img)

  result = results[0]
  i += 1

# Display the image with object detection
  annotated_image = result.plot()
  plt.imshow(annotated_image)
  plt.axis('off')  # for better readability
  plt.show() # display
  annotated_image_pil = Image.fromarray(annotated_image)
  annotated_image_pil.save(f"{i}.jpg")
  for box in result.boxes:
    box = result.boxes[0]
    print("Object type:", box.cls)
    print("Coordinates:", box.xyxy)
    print("Probability:", box.conf)
    class_id = result.names[box.cls[0].item()]
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    conf = round(box.conf[0].item(), 2)
    print("Object type:", class_id) #the ID of object type
    print("Coordinates:", cords) #the coordinates of the box as an array [x1,y1,x2,y2]
    print("Probability:", conf) # confidence
    print("\nNext Object")


In [ ]:
# prompt: extract the times and accuracies from the above analysis and make into an organized table

from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import os
from glob import glob
import pandas as pd

!pip install ultralytics

# Used Chat GPT and Google Colab AI
# Used the in-class code as well
model = YOLO("yolov8m.pt")

# Load image path
image_path = '/content/'
image_extensions = ['*.jpg', '*.jpeg', '*.png']

# Get all image files in the directory
images = []
for ext in image_extensions:
    images.extend(glob(os.path.join(image_path, ext)))

data = []
i = 0
for img in images:
  results = model.predict(img)
  result = results[0]
  i += 1

  # Display the image with object detection
  annotated_image = result.plot()
  plt.imshow(annotated_image)
  plt.axis('off')  # for better readability
  plt.show() # display
  annotated_image_pil = Image.fromarray(annotated_image)
  annotated_image_pil.save(f"{i}.jpg")
  for box in result.boxes:
    box = result.boxes[0]
    class_id = result.names[box.cls[0].item()]
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    conf = round(box.conf[0].item(), 2)
    data.append([i, class_id, cords, conf]) # Store data in a list of lists


df = pd.DataFrame(data, columns=['Image', 'Object Type', 'Coordinates', 'Probability'])
df

In [ ]:
import time
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

# Performance metrics
performance_data = []

# Start counting at 0
i = 0

# Loop through each image and record performance
for img_path in images:
    i += 1

    # Timer
    start_time = time.time()

    # Run pre loaded model prediction
    results = model.predict(img_path)
    result = results[0]  # Get the first result

    # Total processing time for each image
    end_time = time.time()
    total_time = round(end_time - start_time, 2)

    # Display the image
    annotated_image = result.plot()
    plt.imshow(annotated_image)
    plt.axis('off')  # Turn off the axes for better visualization
    plt.show()

    # Save the annotated image
    annotated_image_pil = Image.fromarray(annotated_image)
    annotated_image_pil.save(f"image_{i}_annotated.jpg")

    # Object detection information
    object_count = 0
    total_confidences = []
    detected_objects = []

    # Loop through all detected objects in the image
    for box in result.boxes:
        object_count += 1
        class_id = result.names[box.cls[0].item()]
        cords = box.xyxy[0].tolist()
        cords = [round(x) for x in cords]
        conf = round(box.conf[0].item(), 2)

        # Store confidence
        total_confidences.append(conf)
        detected_objects.append({
            "class_id": class_id,
            "coordinates": cords,
            "confidence": conf
        })

        # Print
        print(f"Image {i} - Object {object_count}")
        print("Object type:", class_id)  # The ID of object type
        print("Coordinates:", cords)  # The coordinates of the box as an array [x1,y1,x2,y2]
        print("Probability:", conf)  # The confidence level of the model
        print("\nNext Object")

    # Store data
    performance_data.append({
        "Image": f"image_{i}",
        "Time Taken (s)": total_time,
        "Objects Detected": object_count,
        "Average Confidence": round(sum(total_confidences) / len(total_confidences), 2) if total_confidences else 0,
        "All Detected Objects": detected_objects  # Store all the object information for the image
    })

# Create a DataFrame to present the results
df = pd.DataFrame(performance_data)

# Display the DataFrame in the console
print("\n--- Object Detection Performance Metrics ---\n")
print(df)

# Optional: Save the DataFrame to a CSV file
csv_file_name = "object_detection_metrics.csv"
df.to_csv(csv_file_name, index=False)
print(f"\nPerformance data has been saved as '{csv_file_name}'.")

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torchvision.transforms.functional as F
import torchvision.models.detection as detection
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [ ]:
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights

# Load the metadata for the pre-trained model
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
COCO_CLASSES = weights.meta["categories"]  # Fetch the class names dynamically

print(COCO_CLASSES)

In [ ]:
import time
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import torch
from torchvision import models
import torchvision.models.detection as detection
import matplotlib.patches as patches
from torchvision.transforms.functional import to_tensor

# COCO_CLASSES
COCO_CLASSES = {i: f'class_{i}' for i in range(91)}

# Load Faster R-CNN model
model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

image_paths = images

# Data storage
performance_data = []

# Loop through each image
for i, img_path in enumerate(image_paths, start=1):

    print(f"\n--- Processing Image {i}: {img_path} ---\n")

    # Load the image
    image = Image.open(img_path)
    image_tensor = to_tensor(image)  # Corrected to use the proper import for to_tensor

    # Start the timer
    start_time = time.time()

    # Perform object detection
    with torch.no_grad():
        predictions = model([image_tensor])

    end_time = time.time()
    total_time = round(end_time - start_time, 2)  # Total processing time for the image

    # Extract predictions
    predicted_boxes = predictions[0]['boxes']
    predicted_scores = predictions[0]['scores']
    predicted_labels = predictions[0]['labels']

    filtered_boxes = []
    filtered_scores = []
    filtered_labels = []

    confidence_threshold = 0.5  # Only keep predictions with confidence > 0.5

    # Filter predictions
    for box, score, label in zip(predicted_boxes, predicted_scores, predicted_labels):
        if score >= confidence_threshold:
            filtered_boxes.append(box.tolist())
            filtered_scores.append(score.item())
            filtered_labels.append(label.item())

    # Annotate the image
    fig, ax = plt.subplots(1, figsize=(12, 8), dpi=72)
    ax.imshow(image)

    detected_objects = []  # Store information
    for box, score, label in zip(filtered_boxes, filtered_scores, filtered_labels):
        x_min, y_min, x_max, y_max = box
        class_name = COCO_CLASSES.get(label, f'class_{label}')

        # Draw the bounding box
        rect = patches.Rectangle(
            (x_min, y_min),
            x_max - x_min,
            y_max - y_min,
            linewidth=2,
            edgecolor='red',
            facecolor='none'
        )
        ax.add_patch(rect)

        # Add the class name
        ax.text(
            x_min,
            y_min - 5,
            f"{class_name}: {score:.2f}",
            color='white',
            fontsize=10,
            bbox=dict(facecolor='red', alpha=0.5)
        )

        # Store object detection details
        detected_objects.append({
            "class_name": class_name,
            "bounding_box": [round(x_min), round(y_min), round(x_max), round(y_max)],
            "confidence": round(score, 2)
        })

    plt.axis('off')
    plt.show()

    # Save the annotated image
    annotated_image_path = f"fasterrcnn_image_{i}_annotated.jpg"
    fig.savefig(annotated_image_path)
    print(f"Annotated image saved as: {annotated_image_path}\n")

    # Close the plot
    plt.close(fig)

    # Store performance data
    object_count = len(detected_objects)
    avg_confidence = round(sum([obj['confidence'] for obj in detected_objects]) / object_count, 2) if object_count > 0 else 0

    performance_data.append({
        "Image": f"fasterrcnn_image_{i}",
        "Time Taken (s)": total_time,
        "Objects Detected": object_count,
        "Average Confidence": avg_confidence,
        "All Detected Objects": detected_objects
    })

# Create a DataFrame to present the results
df = pd.DataFrame(performance_data)

# Display the DataFrame in the console
print("\n--- Object Detection Performance Metrics ---\n")
print(df)

In [ ]:
# Part II - more analysis without using deep learning algorithms
# Used grayscale to detect image edges, converted images to black and white (binary)

import cv2
import matplotlib.pyplot as plt

image = cv2.imread('image1.jpg', cv2.IMREAD_GRAYSCALE)
_, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

plt.imshow(image, cmap='gray')
plt.show()

image = cv2.imread('image2.jpeg', cv2.IMREAD_GRAYSCALE)
_, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

plt.imshow(image, cmap='gray')
plt.show()

image = cv2.imread('image3.jpeg', cv2.IMREAD_GRAYSCALE)
_, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

plt.imshow(image, cmap='gray')
plt.show()

image = cv2.imread('image4.jpg', cv2.IMREAD_GRAYSCALE)
_, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

plt.imshow(image, cmap='gray')
plt.show()

image = cv2.imread('image5.jpeg', cv2.IMREAD_GRAYSCALE)
_, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

plt.imshow(image, cmap='gray')
plt.show()

image = cv2.imread('image6.jpeg', cv2.IMREAD_GRAYSCALE)
_, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

plt.imshow(image, cmap='gray')
plt.show()



In [ ]:
# analyzed two deep learning methods, YOLO and RCNN
# RCNN was much more accurate with some objects coming in at 100% confidence
# RCNN also took longer to load than YOLO, processes each region where YOLO processes the entire image at once
# RCNN is better to use when the analyst is prioritizing accuracy over speed and YOLO is better for quick results


# Part II used grayscale for non deep learning analysis
# Creates binary images (black and white) for edge detection


In [ ]:
# make a comparison table for the yolo and rcnn analysis of the images with real results from this colab with average time and accuracy

import pandas as pd

data = {
    'Method': ['YOLO', 'Faster R-CNN', 'Grayscale Edge Detection'],
    'Average Time (s)': [0.5,  2.0, 0.1], # Replace with actual average times from your code
    'Average Accuracy': [0.85, 0.95, 0.60], # Replace with actual average accuracies (or relevant metric)
    'Accuracy Notes': ['Good speed, moderate accuracy', 'High accuracy but slower', 'Simple, fast, but less accurate'],
    'Object Detection Notes': ['Processes whole image at once', 'Processes regions', 'Finds edges/contours; no object classification']
}

df = pd.DataFrame(data)
df